In [267]:
import pandas as pd
import requests
import yfinance as yf
from bs4 import BeautifulSoup
from pymongo import MongoClient

In [268]:
class Accion:
    def __init__(self, mercado, datosAccion):
        self.nombre = datosAccion[mercado.campo_nombre]
        self.max = datosAccion[mercado.campo_valorMaximo]
        self.min = datosAccion[mercado.campo_valorMinimo]
        self.ult = datosAccion[mercado.campo_UltimoPrecio]
        self.mov = datosAccion[mercado.campo_cambio]
        
class Origen:
    def __init__(self):
       self.__abrirColeccion()

    def __abrirColeccion(self):
        client = MongoClient('localhost')
        bd = client['cotizaciones']
        self.coleccion = bd[self.__class__.__name__]
        
    def _guardarEnBD(self, dataFrame):
        self.coleccion.drop()
        self.coleccion.insert_many(dataFrame.to_dict('records'))
        
    def visualizarBD(self):
        for documento in self.coleccion.find({}):
            print(documento)
            
    def visualizarDF(self):
        df = self.__documentADF(self.coleccion.find({}))
        print (df)

    def __documentADF(self, listColeccion):
        df = pd.DataFrame(list(listColeccion))
        return df.drop(['_id'], axis=1)
        
    def __cotizacionMinima(self):
        pass
    def __cotizacionMaxima(self):
        pass

    def filtrarAcciones(self, nombreAccion):
        accion = self.coleccion.find({self.campo_nombre: {"$in": nombreAccion}})
        return print(self.__documentADF(accion))

    def accion(self, symbol):
        return Accion(self, self.coleccion.find_one({self.campo_nombre: symbol}))

In [269]:
       
class bolsaMadrid(Origen):
    campo_nombre = 'Nombre'
    campo_valorMaximo = 'Máx.'
    campo_valorMinimo = 'Mín.'
    campo_UltimoPrecio = 'Últ.'
    campo_cambio = '% Dif.'
    
    def actualizarBD(self):
        #Hacemos la consulta a la pagina y guardamos el contenido en una variable soup
        url_page = 'https://www.bolsamadrid.es/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000'
        page = requests.get(url_page).text 
        soup = BeautifulSoup(page, "lxml")

        #Buscamos la tabla de donde obtener los datos
        tabla = soup.find('table', attrs={'id': 'ctl00_Contenido_tblAcciones'})

        #creamos una lista vacia que llenaremos con los datos de las tablas
        columnas = []

        #buscamos los encabezados de la tabla y los agregamos a la lista columnos (deberia ubicarse en la posicion 0)
        headers = []
        for th in tabla.find_all('th'):
            headers.append(th.text)
        columnas.append(headers)

        for tr in tabla.find_all('tr'):
            fila = []
            for td in tr.find_all('td'):
                fila.append(td.text)
            if len(fila) > 0:
                columnas.append(fila)
                
        #creamos el dataframe usando la lista columnas,pasamos el contenido (de la posicion 1 en adelante), y lo encabezados en (posicion 0)
        df = pd.DataFrame(columnas[1:], columns=columnas[0])

        #Convertirmos a float
        df = self.__convertirAFloat(df)

        #Guardamos los datos en la base de datos
        self._guardarEnBD(df)
        
    def __convertirAFloat(self, df):
        df["% Dif."] = df["% Dif."].str.replace(",", ".").astype(float)
        df["Máx."] = df["Máx."].str.replace(",", ".").astype(float)
        df["Mín."] = df["Mín."].str.replace(",", ".").astype(float)
        df["Últ."] = df["Últ."].str.replace(",", ".").astype(float)
        return df

nombreDeAcciones =[
    'BBVA', "BBVA.MC", 'BBVA',
    'ELE', "ELE.MC", 'ENDESA',
    'GRLS', 'GRIFOLS CL.A', "GRF.MC",
    'IDR', "IDR.MC", 'INDRA A',
    'REP', "REP.MC", 'REPSOL'
    ]

bolsamadrid = bolsaMadrid()
bolsamadrid.actualizarBD()
#bolsamadrid.visualizarDF()
bolsamadrid.filtrarAcciones(nombreDeAcciones)
accion = bolsamadrid.accion('BBVA')
print(accion.mov)

         Nombre    Últ.  % Dif.    Máx.    Mín.     Volumen  \
0          BBVA   6.279    3.58   6.292   6.030  19.040.383   
1        ENDESA  20.080    0.68  20.080  19.890   2.812.273   
2  GRIFOLS CL.A  19.775   -0.10  20.000  19.675     567.489   
3       INDRA A  10.410   -0.38  10.580  10.340     793.334   
4        REPSOL  11.018   -0.34  11.184  11.000   8.729.704   

  Efectivo (miles €)       Fecha    Hora  
0         118.738,74  01/11/2021  Cierre  
1          56.162,16  01/11/2021  Cierre  
2          11.232,60  01/11/2021  Cierre  
3           8.290,58  01/11/2021  Cierre  
4          96.608,64  01/11/2021  Cierre  
3.58


In [270]:
class yfinance(Origen):
    campo_nombre = 'Nombre'
    campo_valorMaximo = 'Máx.'
    campo_valorMinimo = 'Mín.'
    campo_UltimoPrecio = 'Últ.'
    campo_cambio = '% Dif.'

    def actualizarBD(self, acciones):
        listToStr = ' '.join([str(elem) for elem in acciones])
        data = yf.Tickers(listToStr)
        print(data.tickers)
        #self._guardarEnBD(data)

        """ df_list = list()
        for ticker in acciones:
            data = yf.download(ticker, group_by='ticker', period='1d')
            data['ticker'] = str(ticker)  
            df_list.append(data)

        df = pd.concat(df_list)
        self._guardarEnBD(df) """

yahooFinance = yfinance()
yahooFinance.actualizarBD(nombreDeAcciones)
yahooFinance.visualizarDF()
accion = yahooFinance.accion('BBVA.MC')
print(accion.mov)


{'BBVA': yfinance.Ticker object <BBVA>, 'BBVA.MC': yfinance.Ticker object <BBVA.MC>, 'ELE': yfinance.Ticker object <ELE>, 'ELE.MC': yfinance.Ticker object <ELE.MC>, 'ENDESA': yfinance.Ticker object <ENDESA>, 'GRLS': yfinance.Ticker object <GRLS>, 'GRIFOLS': yfinance.Ticker object <GRIFOLS>, 'CL.A': yfinance.Ticker object <CL.A>, 'GRF.MC': yfinance.Ticker object <GRF.MC>, 'IDR': yfinance.Ticker object <IDR>, 'IDR.MC': yfinance.Ticker object <IDR.MC>, 'INDRA': yfinance.Ticker object <INDRA>, 'A': yfinance.Ticker object <A>, 'REP': yfinance.Ticker object <REP>, 'REP.MC': yfinance.Ticker object <REP.MC>, 'REPSOL': yfinance.Ticker object <REPSOL>}


KeyError: "['_id'] not found in axis"

In [ ]:
class investpy():

   def __init__(self):
      pass

   def extraerDatos():
      pass


class Cotizaciones(bolsaMadrid, yfinance, investpy):

    def __init__(self, acciones):
        self.acciones = acciones
        self.actualizarBD()
       

    def actualizarBD(self):
        bolsaMadrid.obtenerDatos(self)
        yfinance.obtenerDatos(self.acciones)
        ##investpy.obtenerDatos(self)

    def leerDatos():
        pass

In [ ]:
acciones = ['ACCIONA', 'BBVA', 'GRIFOLS CL.A', 'PEPITO', 'REPSOL', 'ENDESA']
cotizaciones = Cotizaciones(acciones)
cotizaciones.mostrarDatosEnPantalla()


AttributeError: type object 'bolsaMadrid' has no attribute 'obtenerDatos'